- This notebook builds some toy roadways
- Can be useful to understand how roadways are built in `AutomotiveDrivingModels` and rendered using `AutoViz`

In [ ]:
using AutomotiveDrivingModels
using AutoViz
using Parameters
using Reel

In [ ]:
include("../src/visualization.jl")

## Maxime roadway: Adapted from `AutonomousMerging.jl`

In [ ]:
# struct and functions: generation of merging roadway
const MAIN_LANE_ID = 1
const MERGE_LANE_ID = 2

"""
    MergingEnvironment
A road network with a main lane and a merging lane. The geometry can be modified by 
passing the parameters as keyword arguments in the constructor
# Parameters
- `lane_width::Float64 = 3.0`
- `main_lane_vmax::Float64 = 15.0`
- `merge_lane_vmax::Float64 = 10.0`
- `main_lane_length::Float64 = 100.0`
- `main_lane_angle::Float64 = float(pi)/4`
- `merge_lane_angle::Float64 = float(pi)/4`
- `merge_lane_length::Float64 = 50.0`
- `after_merge_length::Float64 = 50.0`
# Internals 
- `roadway::Roadway{Float64}` contains all the road segment and lane information
- `merge_point::VecSE2{Float64}` coordinate of the merge point in cartesian frame (0.0, 0.0, 0.0) by default
- `merge_proj::RoadProjection{Int64, Float64}` projection of the merge point on the roadway 
- `merge_index::RoadIndex`
"""
@with_kw struct MergingEnvironment
    lane_width::Float64 = 3.0
    main_lane_vmax::Float64 = 15.0
    merge_lane_vmax::Float64 = 10.0
    main_lane_length::Float64 = 100.0
    main_lane_angle::Float64 = float(pi)/4
    merge_lane_angle::Float64 = float(pi)/4
    merge_lane_length::Float64 = 50.0
    after_merge_length::Float64 = 50.0

    # internals
    roadway::Roadway{Float64} = generate_merging_roadway(lane_width, 
                                                main_lane_vmax, 
                                                merge_lane_vmax, 
                                                main_lane_length,
                                                merge_lane_length,
                                                after_merge_length, 
                                                main_lane_angle, 
                                                merge_lane_angle) 
    merge_point::VecSE2{Float64} = VecSE2(0.0, 0.0, 0.0)
    merge_proj::RoadProjection{Int64, Float64} = proj(merge_point, roadway)
    merge_index::RoadIndex{Int64, Float64} = RoadIndex(merge_proj.curveproj.ind, merge_proj.tag)
end


function append_to_curve!(target::Curve{T}, newstuff::Curve{T}) where T <: Real
    s_end = target[end].s
    for c in newstuff
        push!(target, CurvePt{T}(c.pos, c.s+s_end, c.k, c.kd))
    end
    return target
end

"""
    generate_merging_roadway(lane_width::Float64 = 3.0, main_lane_vmax::Float64 = 20.0, 
merge_lane_vmax::Float64 = 15.0, main_lane_length::Float64 = 20.0, 
merge_lane_length::Float64 = 20.0, after_merge_length::Float64 = 20.0, 
main_lane_angle::Float64 = float(pi)/4, merge_lane_angle::Float64 = float(pi)/4)
Generate a `Roadway` object representing a merging scenario. 
The merge point is defined at (0., 0.) by default.
"""
function generate_merging_roadway(lane_width::Float64 = 3.0, 
                                   main_lane_vmax::Float64 = 20.0,
                                   merge_lane_vmax::Float64 = 15.0,
                                   main_lane_length::Float64 = 20.0, 
                                   merge_lane_length::Float64 = 20.0,
                                   after_merge_length::Float64 = 20.0,
                                   main_lane_angle::Float64 = float(pi)/4, 
                                   merge_lane_angle::Float64 = float(pi)/4) 
    # init empty roadway 
    roadway = Roadway()
    n_pts = 2 # sample points for the roadway, only two needed each time, since all segments are straight
    main_tag = LaneTag(MAIN_LANE_ID, 1)
    merge_tag = LaneTag(MERGE_LANE_ID, 1)
    # after_merge_tag = LaneTag(AFTER_MERGE_LANE_ID, 1)

    # define curves
    merge_point = VecE2(0.0, 0.0) 
    main_lane_startpt = merge_point + polar(main_lane_length, -float(pi) - main_lane_angle)
    main_curve = gen_straight_curve(main_lane_startpt, merge_point, n_pts)
    merge_index = curveindex_end(main_curve)
    append_to_curve!(main_curve, 
        gen_straight_curve(merge_point, merge_point + polar(after_merge_length, 0.0), n_pts)[2:end])
    merge_lane_startpt = merge_point + polar(merge_lane_length, float(pi) + merge_lane_angle)
    merge_curve = gen_straight_curve(merge_lane_startpt, merge_point, n_pts)


    # define lanes with connections 
    main_lane = Lane(main_tag, main_curve, width = lane_width, speed_limit=SpeedLimit(0.,main_lane_vmax))
    merge_lane = Lane(merge_tag, merge_curve, width = lane_width,speed_limit=SpeedLimit(0.,merge_lane_vmax),
                        next=RoadIndex(merge_index, main_tag))

    # add segments to roadway 
    push!(roadway.segments, RoadSegment(MAIN_LANE_ID, [main_lane]))
    push!(roadway.segments, RoadSegment(MERGE_LANE_ID, [merge_lane]))
  
    return roadway
end

In [ ]:
scene = Scene()
state1 = VehicleState(Frenet(roadway[LaneTag(1,1)],3.0), roadway, 10.0)
veh1 = Vehicle(state1, VehicleDef(), 1)

push!(scene, veh1)

In [ ]:
cpt_o = curvepts_overlay(roadway,colorant"0xFFEF00",0.3)
render(scene,roadway,[cpt_o],cam=FitToContentCamera(0.2))

In [ ]:
# roadway creation: make a merging roadway
roadway2 = generate_merging_roadway(3.,20.,15.,40.,40.,40.,0.,-float(0.1));
cpt_o = curvepts_overlay(roadway2,colorant"0xFFEF00",0.3)
scene2 = Scene()
state2 = VehicleState(Frenet(roadway2[LaneTag(1,1)],13.0), roadway2, 10.0)
veh1 = Vehicle(state2, VehicleDef(), 1)

state3 = VehicleState(Frenet(roadway2[LaneTag(2,1)],5.0),roadway2,10.0)
veh2 = Vehicle(state3,VehicleDef(),2)
push!(scene2, veh1)
push!(scene2,veh2)
render(scene2,roadway2,[cpt_o],cam=FitToContentCamera(0.2))

In [ ]:
scene_list = get_hallucination_scenes(scene2,models=models,traj=int_trajdata,roadway=roadway2)

In [ ]:
scenelist2video(scene_list,roadway=roadway2,filename="media/interaction_vids/merge.mp4")

## William Brannon's roadway

In [ ]:
# WBs example code for roadway generation: function and then script
function get_track_wb()

    x_coods = [0., 20.]
    y_coods = [3., 3.]
    theta = [0., 0.]
    
    turn_length = 10
    new_theta = 0.
    incremental_angle = (pi/4) / turn_length
    radius = 1
    for i = 1:turn_length
        push!(theta, new_theta)
        push!(x_coods, x_coods[end] + radius * cos(new_theta))
        push!(y_coods, y_coods[end] + radius * sin(new_theta))
        new_theta -= (incremental_angle)
    end

    # x coods in first row, y coods in second row, theta in third row
    mid_coods = vcat(x_coods', y_coods', theta') 
    
    
    first_cood = VecSE2(mid_coods[1,1], mid_coods[2,1], mid_coods[3,1])
    second_cood = VecSE2(mid_coods[1,2], mid_coods[2,2], mid_coods[3,2])
    radius = 0.01
    nsamples = 20

    track = gen_bezier_curve(first_cood, second_cood, radius, radius, nsamples)
    
    curve_radius = incremental_angle
    nsamples = 1000
    for i = 3:turn_length+2
        turn1 = VecSE2(mid_coods[1, i-1], mid_coods[2, i-1], mid_coods[3, i-1])
        turn2 = VecSE2(mid_coods[1, i], mid_coods[2, i], mid_coods[3, i])
        curve = gen_bezier_curve(turn1, turn2, curve_radius, curve_radius, nsamples)
        append_to_curve!(track, curve)
    end

    return track
end

# script: generate WB's roadway
nlanes = 1


roadway_wb = gen_straight_roadway(nlanes, 50.)

#width = DEFAULT_LANE_WIDTH
#roadway = Roadway()

track_wb = get_track_wb()

lane = Lane(LaneTag(2, 1), track_wb, width=DEFAULT_LANE_WIDTH)
push!(roadway_wb.segments, RoadSegment(2, [lane]))
cam = FitToContentCamera(0.05)
render(roadway_wb, cam=cam)

In [ ]:
cpt_o_wb = curvepts_overlay(roadway_wb,colorant"0xFFEF00",0.3)
scene_wb = Scene()
state_wb = VehicleState(Frenet(roadway_wb[LaneTag(1,1)],13.0), roadway_wb, 10.0)
veh1 = Vehicle(state_wb, VehicleDef(), 1)
l2 = roadway_wb.segments[2].lanes[1]
state_wb2 = VehicleState(Frenet(roadway_wb[LaneTag(2,1)],5.0),roadway_wb,10.0)
@show "here2"
veh2 = Vehicle(state_wb2,VehicleDef(),2)
push!(scene_wb, veh1)
push!(scene_wb,veh2)
render(scene_wb,roadway_wb,[cpt_o_wb],cam=FitToContentCamera(0.2))

In [ ]:
scene_list_wb = get_hallucination_scenes(scene_wb,duration=5,models=models,
    traj=int_trajdata,roadway=roadway_wb)
scenelist2video(scene_list_wb,roadway=roadway_wb,filename="media/interaction_vids/merge_wb.mp4")

## Driving simulation functions

In [ ]:
# models creation: 2d driver model for the merging roadway
models = Dict{Int64,DriverModel}()
for veh in scene2
    models[veh.id] = Tim2DDriver(0.1,mlane=MOBIL(INTERACTION_TIMESTEP,mlon=IntelligentDriverModel()))
end

In [ ]:
# function: get hallucination scenes
"""
    function get_hallucination_scenes
- Hallucinate starting from `start_step` for `nsteps` using `models` and return a list of scenes
- Used by `plot_carwise_pos_vel` to assess position and velocity traces against ground truth

# Returns
- `halluc_scenes_list`: List containing the scenes starting with the ground truth scene at `start_step`

# Examples
```julia
scene_list = get_hallucination_scenes(start_scene,nsteps=100,models=models);
```
"""
function get_hallucination_scenes(scene_halluc;models,start_step=1,duration=3,id_list=[],
        traj=traj_interaction,verbosity = false,timestep=INTERACTION_TIMESTEP,roadway=roadway_interaction)
        # Setting up
    halluc_scenes_list = []
    #scene_halluc = get_scene(start_step,traj) # Frame to start hallucination from
    push!(halluc_scenes_list,deepcopy(scene_halluc))
    
    nsteps = duration/timestep
    for (i,t) in enumerate(start_step:start_step+nsteps-1)
        
        if !isempty(id_list) keep_vehicle_subset!(scene_halluc,id_list) end
        
        actions = Array{Any}(undef,length(scene_halluc))

            # Propagation of scene forward
        get_actions!(actions,scene_halluc,roadway,models)

        tick!(scene_halluc,roadway,actions,timestep)
        
        push!(halluc_scenes_list,deepcopy(scene_halluc))
    end 
    return halluc_scenes_list
end